# Testing on Test Dataset

In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import librosa
import os
import math
import shutil
import pandas as pd
import random
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import LSTM


2025-07-05 08:39:05.831361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751704746.100272      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751704746.186635      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.load_model('/kaggle/input/cnnlstmspoof/tensorflow2/v2/2/audio_cnn_lstm (2).h5')

2025-07-05 08:39:22.002756: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
model.input_shape[1]

128

In [4]:
SAMPLE_RATE = 16000  # Sample rate of your audio files
DURATION = 4  # Duration of audio clips in seconds
N_MELS = model.input_shape[1]  # Number of Mel frequency bins
EER_THRESHOLD = 0.2104

In [5]:
import os
import math
import json
import librosa
import numpy as np

max_time_steps = 109  # Define the maximum time steps for your model

def save_mel(train_df):
    X = []
    y = []
    for _, row in train_df.iterrows():
        file_path = row["filepath"]
        label = int(row["target"])

        try:
            # Load audio file using librosa
            audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
        
            # Extract Mel spectrogram using librosa
            mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS)
            mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        
            # Ensure all spectrograms have the same width (time steps)
            if mel_spectrogram.shape[1] < max_time_steps:
                mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, max_time_steps - mel_spectrogram.shape[1])), mode='constant')
            else:
                mel_spectrogram = mel_spectrogram[:, :max_time_steps]
        
            X.append(mel_spectrogram)
            y.append(label)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    # Save JSON
    # with open(json_path, "w") as fp:
    #     json.dump(data, fp, indent=4)
    return X,y


In [6]:
### for creating folders for eval and dev
BASE_PATH = '/kaggle/input/asvpoof-2019-dataset/LA/LA'

# change here
eval_df = pd.read_csv(f'{BASE_PATH}/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt', sep=" ", header=None)

eval_df.columns =['speaker_id','filename','system_id','null','class_name']
eval_df.drop(columns=['null'],inplace=True)

#change here
eval_df['filepath'] = f'{BASE_PATH}/ASVspoof2019_LA_eval/flac/'+eval_df.filename+'.flac'

eval_df['target'] = (eval_df.class_name=='spoof').astype('int32')
print('len eval', len(eval_df))
eval_df.head()

len eval 71237


,speaker_id,filename,system_id,class_name,filepath,target
0,LA_0039,LA_E_2834763,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
1,LA_0014,LA_E_8877452,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
2,LA_0040,LA_E_6828287,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
3,LA_0022,LA_E_6977360,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
4,LA_0031,LA_E_5932896,-,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1


In [7]:
BASE_PATH = '/kaggle/input/asvpoof-2019-dataset/LA/LA'

# change here
dev_df = pd.read_csv(f'{BASE_PATH}/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt', sep=" ", header=None)

dev_df.columns =['speaker_id','filename','system_id','cat','class_name']

#change here
dev_df['filepath'] = f'{BASE_PATH}/ASVspoof2019_LA_dev/flac/'+dev_df.filename+'.flac'

dev_df['target'] = (dev_df.class_name=='spoof').astype('int32')
print('len dev', len(dev_df))
dev_df.head()

len dev 24844


,speaker_id,filename,system_id,cat,class_name,filepath,target
0,LA_0069,LA_D_1047731,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
1,LA_0069,LA_D_1105538,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
2,LA_0069,LA_D_1125976,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
3,LA_0069,LA_D_1293230,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
4,LA_0069,LA_D_1340209,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0


In [8]:
bonafide_df = eval_df[eval_df["target"] == 0]
spoof_df = eval_df[eval_df["target"] == 1].sample(n=len(bonafide_df), random_state=42)
balanced_df = pd.concat([bonafide_df, spoof_df], ignore_index=True).sample(frac=1, random_state=42)
print(len(balanced_df))
eval_df= balanced_df

14710


In [9]:
bonafide_df_dev = dev_df[dev_df["target"] == 0]
spoof_df_dev = dev_df[dev_df["target"] == 1].sample(n=len(bonafide_df), random_state=42)
balanced_df_dev = pd.concat([bonafide_df, spoof_df], ignore_index=True).sample(frac=1, random_state=42)
print(len(balanced_df_dev))
dev_df= balanced_df_dev

14710


In [10]:
# eval_sample = eval_df.sample(n=3000, random_state=42).reset_index(drop=True)
eval_sample = pd.concat([dev_df, eval_df], ignore_index=True).sample(frac=1, random_state=42)
print(len(eval_sample))

# Step 2: Run your feature extraction function
X_test1, y_test1 = save_mel(eval_sample)

29420


In [11]:
X_test1 = np.array(X_test1)
y_test1 = np.array(y_test1)

In [12]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score, accuracy_score

In [13]:
test_loss, test_acc, test_recall = model.evaluate(X_test1, y_test1, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Recall: {test_recall:.4f}")

920/920 - 240s - 261ms/step - accuracy: 0.8835 - loss: 0.3751 - recall: 0.7792
Test Loss: 0.3751
Test Accuracy: 0.8835
Test Recall: 0.7792


In [14]:
from sklearn.metrics import f1_score

X_test1 = np.expand_dims(X_test1, axis=-1)
# Step 1: Get predicted probabilities
y_probs = model.predict(X_test1, verbose=1).flatten()

# Step 2: Convert to binary predictions (e.g. threshold at 0.5 or your EER threshold)
threshold = 0.5
y_preds = (y_probs >= EER_THRESHOLD).astype(int)

# Step 3: Calculate F1 score
f1 = f1_score(y_test1, y_preds)
precision = precision_score(y_test1, y_preds)
auc = roc_auc_score(y_test1, y_probs)  
acc = accuracy_score(y_test1, y_preds)
recall = recall_score(y_test1, y_preds)

print(f"F1 Score: {f1:.4f}")

920/920 ━━━━━━━━━━━━━━━━━━━━ 223s 241ms/step
F1 Score: 0.8738


In [15]:
from sklearn.metrics import roc_curve

# Step 2: Get ROC curve
fpr, tpr, thresholds = roc_curve(y_test1, y_probs)
fnr = 1 - tpr

# Step 3: Find the EER (where FPR and FNR are closest)
eer_index = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_index]
eer_threshold = thresholds[eer_index]

print(f"EER: {eer:.4f}")
print(f"EER Threshold: {eer_threshold:.4f}")


EER: 0.1398
EER Threshold: 0.1295


In [16]:
import json
results_final = {
        'EER': round(eer, 4),
        'F1 Score': round(f1, 4),
        'Accuracy': round(acc, 4),
        'AUC': round(auc, 4),
        'Recall': round(test_recall, 4),
        'Precision': round(precision, 4),
        'Samples': len(X_test1)
}

with open("results_final.json", "w") as f:
    json.dump(results_final, f, indent=4)  # `indent=4` for pretty formatting

results_final

{'EER': 0.1398,
 'F1 Score': 0.8738,
 'Accuracy': 0.8797,
 'AUC': 0.931,
 'Recall': 0.7792,
 'Precision': 0.9191,
 'Samples': 29420}

# Final Testing

In [17]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import librosa
import os
import math
import shutil
import pandas as pd
import random

In [18]:
BASE_PATH = '/kaggle/input/asvpoof-2019-dataset/LA/LA'
eval_df = pd.read_csv(f'{BASE_PATH}/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt', sep=" ", header=None)

eval_df.columns =['speaker_id','filename','system_id','cat','class_name']

#change here
eval_df['filepath'] = f'{BASE_PATH}/ASVspoof2019_LA_eval/flac/'+eval_df.filename+'.flac'

eval_df['target'] = (eval_df.class_name=='spoof').astype('int32')
print('len eval', len(eval_df))
eval_df.head()

len eval 71237


,speaker_id,filename,system_id,cat,class_name,filepath,target
0,LA_0039,LA_E_2834763,-,A11,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
1,LA_0014,LA_E_8877452,-,A14,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
2,LA_0040,LA_E_6828287,-,A16,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
3,LA_0022,LA_E_6977360,-,A09,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1
4,LA_0031,LA_E_5932896,-,A13,spoof,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,1


In [19]:
import pandas as pd

# Count spoof and bonafide samples
spoof_df = eval_df[eval_df['target'] == 1]
bonafide_df = eval_df[eval_df['target'] == 0]
print(len(spoof_df))

63882


In [20]:
SAMPLE_RATE = 16000
TRACK_DURATION = 4 
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
EER_THRESHOLD = 0.2104

In [21]:
def test_file(model, file_path, sample_rate=16000, duration=4.0, n_mels=N_MELS, max_time_steps=109, eer_threshold=EER_THRESHOLD):
    X = []

    try:
        # Load audio
        audio, _ = librosa.load(file_path, sr=sample_rate, duration=duration)

        # Compute Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)  # (n_mels, time)

        # Pad or truncate to fixed time steps
        if mel_spec.shape[1] < max_time_steps:
            pad_width = max_time_steps - mel_spec.shape[1]
            mel_spec = np.pad(mel_spec, ((0, 0), (0, pad_width)), mode='constant')
        else:
            mel_spec = mel_spec[:, :max_time_steps]

        X.append(mel_spec)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return -1, -1

    # Convert to model input shape: (1, 40, 109, 1)
    X = np.array(X)
    X = np.expand_dims(X, axis=-1)

    # Predict
    prob = model.predict(X, verbose=0)
    prob_label = int(prob[0][0] >= eer_threshold)

    return prob[0][0], prob_label


In [22]:
from sklearn.metrics import f1_score, accuracy_score, roc_curve
import numpy as np

# Constants required by save_mel
SAMPLE_RATE = 16000
DURATION = 4.0
N_MELS = 128
max_time_steps = 109

# Function to compute EER
def compute_eer(y_true, y_scores):
    if len(np.unique(y_true)) < 2:
        print("EER cannot be computed — only one class present in y_true.")
        eer = 0
        eer_threshold = None
    else:
        fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = fpr[eer_index]
        eer_threshold = thresholds[eer_index]
        print(f"EER: {eer:.4f}")
        print(f"EER Threshold: {eer_threshold:.4f}")
    return eer

# Dictionary to store results
results_by_cat1 = {}
spoof_eval_df = eval_df[eval_df["target"] == 1]

# Loop through each category
for cat in sorted(spoof_eval_df["cat"].unique()):
    print(f"Processing category {cat}...")
    
    df_cat = eval_df[eval_df["cat"] == cat].reset_index(drop=True)
    bonafide_samples = eval_df[eval_df['target'] == 0]
    num_spoof = len(df_cat)
    num_bonafide = len(bonafide_samples)
    n_samples = min(num_bonafide, num_spoof)
    bonafide_samples = bonafide_samples.sample(n=n_samples, random_state=42)
    spoof_samples = df_cat.sample(n=n_samples, random_state=42)  # Downsample spoof too
    df_cat = pd.concat([spoof_samples, bonafide_samples]).sample(frac=1, random_state=42).reset_index(drop=True)


    # Batch mel extraction
    X_batch, y_batch = save_mel(df_cat)

    if len(X_batch) == 0:
        print(f"No valid samples in category {cat}")
        continue

    # Convert to numpy and reshape for model
    X_batch = np.array(X_batch)
    X_batch = np.expand_dims(X_batch, axis=-1)  # Shape: (num_samples, 40, 109, 1)
    y_batch = np.array(y_batch)

    # Predict probabilities
    y_probs = model.predict(X_batch, verbose=0).flatten()
    y_preds = (y_probs >= EER_THRESHOLD).astype(int)

    # Compute metrics
    eer = compute_eer(y_batch, y_probs)
    f1 = f1_score(y_batch, y_preds)
    acc = accuracy_score(y_batch, y_preds)
    auc = roc_auc_score(y_batch, y_probs)           # Use y_probs here, not y_preds
    recall = recall_score(y_batch, y_preds)
    precision = precision_score(y_batch, y_preds)


    # Store results
    results_by_cat1[cat] = {
        'EER': round(eer, 4),
        'F1 Score': round(f1, 4),
        'Accuracy': round(acc, 4),
        'AUC': round(auc, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'Samples': len(y_batch)
    }

# Print results
for cat, metrics in results_by_cat1.items():
    print(f"{cat}: {metrics}")


Processing category A07...
EER: 0.0014
EER Threshold: 0.9772
Processing category A08...
EER: 0.1530
EER Threshold: 0.1172
Processing category A09...
EER: 0.0008
EER Threshold: 0.9950
Processing category A10...
EER: 0.0026
EER Threshold: 0.8730
Processing category A11...
EER: 0.0033
EER Threshold: 0.8525
Processing category A12...
EER: 0.0022
EER Threshold: 0.9171
Processing category A13...
EER: 0.0045
EER Threshold: 0.8049
Processing category A14...
EER: 0.0016
EER Threshold: 0.9765
Processing category A15...
EER: 0.0049
EER Threshold: 0.7802
Processing category A16...
EER: 0.0096
EER Threshold: 0.5527
Processing category A17...
EER: 0.4796
EER Threshold: 0.0362
Processing category A18...
EER: 0.1827
EER Threshold: 0.1018
Processing category A19...
EER: 0.3067
EER Threshold: 0.0616
A07: {'EER': 0.0014, 'F1 Score': 0.9648, 'Accuracy': 0.9635, 'AUC': 0.9996, 'Recall': 1.0, 'Precision': 0.9319, 'Samples': 9828}
A08: {'EER': 0.153, 'F1 Score': 0.8664, 'Accuracy': 0.8735, 'AUC': 0.9278, 'Re

In [23]:
BASE_PATH = '/kaggle/input/asvpoof-2019-dataset/LA/LA'

# change here
dev_df = pd.read_csv(f'{BASE_PATH}/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt', sep=" ", header=None)

dev_df.columns =['speaker_id','filename','system_id','cat','class_name']

#change here
dev_df['filepath'] = f'{BASE_PATH}/ASVspoof2019_LA_dev/flac/'+dev_df.filename+'.flac'

dev_df['target'] = (dev_df.class_name=='spoof').astype('int32')
print('len dev', len(dev_df))
dev_df.head()

len dev 24844


,speaker_id,filename,system_id,cat,class_name,filepath,target
0,LA_0069,LA_D_1047731,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
1,LA_0069,LA_D_1105538,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
2,LA_0069,LA_D_1125976,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
3,LA_0069,LA_D_1293230,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0
4,LA_0069,LA_D_1340209,-,-,bonafide,/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVsp...,0


In [24]:
import pandas as pd

# Count spoof and bonafide samples
spoof_df = dev_df[dev_df['target'] == 1]
bonafide_df = dev_df[dev_df['target'] == 0]

min_len = min(len(spoof_df), len(bonafide_df))
print(len(spoof_df))
print(len(bonafide_df))

22296
2548


In [25]:
# Dictionary to store results
results_by_cat2 = {}
spoof_dev_df = dev_df[dev_df["target"] == 1]
# Loop through each category
for cat in sorted(spoof_dev_df["cat"].unique()):
    print(f"Processing category {cat}...")
    
    df_cat = dev_df[dev_df["cat"] == cat].reset_index(drop=True)
    bonafide_samples = dev_df[dev_df['target'] == 0]
    num_spoof = len(df_cat)
    num_bonafide = len(bonafide_samples)
    n_samples = min(num_bonafide, num_spoof)
    bonafide_samples = bonafide_samples.sample(n=n_samples, random_state=42)
    spoof_samples = df_cat.sample(n=n_samples, random_state=42)  # Downsample spoof too
    df_cat = pd.concat([spoof_samples, bonafide_samples]).sample(frac=1, random_state=42).reset_index(drop=True)


    # Batch mel extraction
    X_batch, y_batch = save_mel(df_cat)

    if len(X_batch) == 0:
        print(f"No valid samples in category {cat}")
        continue

    # Convert to numpy and reshape for model
    X_batch = np.array(X_batch)
    X_batch = np.expand_dims(X_batch, axis=-1)  # Shape: (num_samples, 40, 109, 1)
    y_batch = np.array(y_batch)

    # Predict probabilities
    y_probs = model.predict(X_batch, verbose=0).flatten()
    y_preds = (y_probs >= EER_THRESHOLD).astype(int)

    # Compute metrics
    eer = compute_eer(y_batch, y_probs)
    f1 = f1_score(y_batch, y_preds)
    acc = accuracy_score(y_batch, y_preds)
    auc = roc_auc_score(y_batch, y_probs)           # Use y_probs here, not y_preds
    recall = recall_score(y_batch, y_preds)
    precision = precision_score(y_batch, y_preds)

    # Store results
    results_by_cat2[cat] = {
        'EER': round(eer, 4),
        'F1 Score': round(f1, 4),
        'Accuracy': round(acc, 4),
        'AUC': round(auc, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'Samples': len(y_batch)
    }

# Print results
for cat, metrics in results_by_cat2.items():
    print(f"{cat}: {metrics}")


Processing category A01...
EER: 0.0027
EER Threshold: 0.9131
Processing category A02...
EER: 0.0008
EER Threshold: 0.9973
Processing category A03...
EER: 0.0012
EER Threshold: 0.9908
Processing category A04...
EER: 0.0067
EER Threshold: 0.7115
Processing category A05...
EER: 0.0177
EER Threshold: 0.4455
Processing category A06...
EER: 0.1692
EER Threshold: 0.1194
A01: {'EER': 0.0027, 'F1 Score': 0.9644, 'Accuracy': 0.9631, 'AUC': 0.9996, 'Recall': 1.0, 'Precision': 0.9313, 'Samples': 5096}
A02: {'EER': 0.0008, 'F1 Score': 0.9644, 'Accuracy': 0.9631, 'AUC': 0.9996, 'Recall': 1.0, 'Precision': 0.9313, 'Samples': 5096}
A03: {'EER': 0.0012, 'F1 Score': 0.9644, 'Accuracy': 0.9631, 'AUC': 0.9996, 'Recall': 1.0, 'Precision': 0.9313, 'Samples': 5096}
A04: {'EER': 0.0067, 'F1 Score': 0.9629, 'Accuracy': 0.9615, 'AUC': 0.9989, 'Recall': 0.9969, 'Precision': 0.9311, 'Samples': 5096}
A05: {'EER': 0.0177, 'F1 Score': 0.9593, 'Accuracy': 0.958, 'AUC': 0.9961, 'Recall': 0.9898, 'Precision': 0.9306, '

In [26]:
results_by_cat1

{'A07': {'EER': 0.0014,
  'F1 Score': 0.9648,
  'Accuracy': 0.9635,
  'AUC': 0.9996,
  'Recall': 1.0,
  'Precision': 0.9319,
  'Samples': 9828},
 'A08': {'EER': 0.153,
  'F1 Score': 0.8664,
  'Accuracy': 0.8735,
  'AUC': 0.9278,
  'Recall': 0.8201,
  'Precision': 0.9182,
  'Samples': 9828},
 'A09': {'EER': 0.0008,
  'F1 Score': 0.9648,
  'Accuracy': 0.9635,
  'AUC': 0.9999,
  'Recall': 1.0,
  'Precision': 0.9319,
  'Samples': 9828},
 'A10': {'EER': 0.0026,
  'F1 Score': 0.9629,
  'Accuracy': 0.9616,
  'AUC': 0.9984,
  'Recall': 0.9963,
  'Precision': 0.9317,
  'Samples': 9828},
 'A11': {'EER': 0.0033,
  'F1 Score': 0.9627,
  'Accuracy': 0.9614,
  'AUC': 0.9983,
  'Recall': 0.9959,
  'Precision': 0.9317,
  'Samples': 9828},
 'A12': {'EER': 0.0022,
  'F1 Score': 0.9648,
  'Accuracy': 0.9635,
  'AUC': 0.9995,
  'Recall': 1.0,
  'Precision': 0.9319,
  'Samples': 9828},
 'A13': {'EER': 0.0045,
  'F1 Score': 0.9648,
  'Accuracy': 0.9635,
  'AUC': 0.9991,
  'Recall': 1.0,
  'Precision': 0.931

In [27]:
results_by_cat2

{'A01': {'EER': 0.0027,
  'F1 Score': 0.9644,
  'Accuracy': 0.9631,
  'AUC': 0.9996,
  'Recall': 1.0,
  'Precision': 0.9313,
  'Samples': 5096},
 'A02': {'EER': 0.0008,
  'F1 Score': 0.9644,
  'Accuracy': 0.9631,
  'AUC': 0.9996,
  'Recall': 1.0,
  'Precision': 0.9313,
  'Samples': 5096},
 'A03': {'EER': 0.0012,
  'F1 Score': 0.9644,
  'Accuracy': 0.9631,
  'AUC': 0.9996,
  'Recall': 1.0,
  'Precision': 0.9313,
  'Samples': 5096},
 'A04': {'EER': 0.0067,
  'F1 Score': 0.9629,
  'Accuracy': 0.9615,
  'AUC': 0.9989,
  'Recall': 0.9969,
  'Precision': 0.9311,
  'Samples': 5096},
 'A05': {'EER': 0.0177,
  'F1 Score': 0.9593,
  'Accuracy': 0.958,
  'AUC': 0.9961,
  'Recall': 0.9898,
  'Precision': 0.9306,
  'Samples': 5096},
 'A06': {'EER': 0.1692,
  'F1 Score': 0.7772,
  'Accuracy': 0.8044,
  'AUC': 0.909,
  'Recall': 0.6825,
  'Precision': 0.9024,
  'Samples': 5096}}

In [28]:
with open("dev.json", "w") as f:
    json.dump(results_by_cat2, f, indent=4)  # `indent=4` for pretty formatting

In [29]:
with open("eval.json", "w") as f:
    json.dump(results_by_cat1, f, indent=4)  # `indent=4` for pretty formatting